In [ ]:
# Load data
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

path1 = 'Sign1_full_fitted.csv'
data1 = pd.read_csv(path1, usecols=[0,1], header=0)
data1['ts'] = pd.to_datetime(data1['ts'])
data1['unique_id'] = 'garage1'
data1 = data1.rename(columns={'ts': 'ds', 'y1': 'y'})
print(data1)
data1.plot(x = 'ds', y = 'y')

In [ ]:
path2 = 'Sign12_full_fitted.csv'
data2 = pd.read_csv(path2, usecols=[0,1], header=0)
data2['ts'] = pd.to_datetime(data2['ts'])
data2['unique_id'] = 'garage2'
data2 = data2.rename(columns={'ts': 'ds', 'y12': 'y'})
print(data2)
data2.plot(x = 'ds', y = 'y')

In [ ]:
path3 = 'Sign14_full_fitted.csv'
data3 = pd.read_csv(path3, usecols=[0,1], header=0)
data2['ts'] = pd.to_datetime(data3['ts'])
data3['unique_id'] = 'garage3'
data3 = data3.rename(columns={'ts': 'ds', 'y14': 'y'})
print(data3)
data3.plot(x = 'ds', y = 'y')

In [ ]:
df = pd.concat([data1, data2, data3])
df['ds'] = pd.to_datetime(df['ds'])
df = df.drop('ts', axis=1)

train = df.loc[df['ds'] < '2016-11-17']
valid = df.loc[(df['ds'] >= '2016-11-17')]

h = valid['ds'].nunique()

In [ ]:
from lightgbm import LGBMRegressor
from mlforecast import MLForecast

from window_ops.rolling import rolling_mean, rolling_max, rolling_min

models = [LGBMRegressor(random_state=0, n_estimators=100)]

model = MLForecast(models=models,
                   freq='T',
                   lags=[1440],
                   date_features=['dayofweek'],
                   )

model.fit(train, id_col='unique_id', time_col='ds', target_col='y', static_features=[])

p = model.predict(horizon=h)
p = p.merge(valid[['unique_id', 'ds', 'y']], on=['unique_id', 'ds'], how='left')

from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(p['y'], p['LGBMRegressor'])

In [ ]:
from utilsforecast.plotting import plot_series
fig = plot_series(train, p, max_ids=4, plot_random=False)
fig

In [ ]:
import optuna

def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1)
    num_leaves = trial.suggest_int('num_leaves', 2, 256)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 100)
    bagging_fraction  = trial.suggest_float('bagging_fraction', 0.1, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 1.0)
    lags = trial.suggest_int('lags', 14, 56, step=7)

    models = [LGBMRegressor(random_state=0, n_estimators=500, bagging_freq=1,
                            learning_rate=learning_rate, num_leaves=num_leaves,
                            min_data_in_leaf=min_data_in_leaf, bagging_fraction=bagging_fraction,
                            colsample_bytree=colsample_bytree)]
                            

    model = MLForecast(models=models,
                    freq='T',
                    lags=[lags],
                    date_features=['dayofweek']
                    )


    model.fit(train, id_col='unique_id', time_col='ds', target_col='y', static_features=[])

    p = model.predict(horizon=h)
    p = p.merge(valid[['unique_id', 'ds', 'y']], on=['unique_id', 'ds'], how='left')

    error = mean_absolute_percentage_error(p['y'], p['LGBMRegressor'])
    
    return error


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

In [ ]:
study.best_params

In [ ]:
study.best_value